In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install Kaggle
! pip install -q kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"giuseppevenuto","key":"b789d1bcaee3e95873e6a9f9533a5ec3"}'}

In [3]:
# Move the Kaggle API Token in the correct folder, test it works
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                         480KB  2023-01-04 12:50:31           8316        288  1.0              
googleai/musiccaps                                              MusicCaps                                          793KB  2023-01-25 09:25:48           1509        161  0.9411765        
themrityunjaypathak/most-subscribed-1000-youtube-channels       Most Subscribed 1000 Youtube Channels               28KB  2023-01-21 14:42:05           1965         63  1.0              
nitishsharma01/olympics-124-years-datasettill-2020              O

In [4]:
# Download the dataset from Kaggle
! kaggle datasets download samlearner/letterboxd-movie-ratings-data

 94% 177M/188M [00:02<00:00, 68.7MB/s]
100% 188M/188M [00:02<00:00, 83.0MB/s]


In [5]:
# Unzip the data
! unzip letterboxd-movie-ratings-data.zip -d dataset

Archive:  letterboxd-movie-ratings-data.zip
  inflating: dataset/movie_data.csv  
  inflating: dataset/ratings_export.csv  
  inflating: dataset/users_export.csv  


In [6]:
import numpy as np
import pandas as pd

In [7]:
# Load the dataset into a Pandas dataframe
movie_data = pd.read_csv("dataset/movie_data.csv", lineterminator="\n")
ratings_data = pd.read_csv("dataset/ratings_export.csv", lineterminator="\n")
user_data = pd.read_csv("dataset/users_export.csv", lineterminator="\n")

In [8]:
import pickle

def get_and_save_or_load_maps(d):
  mm = "./processed/movie_to_index.pkl"
  uu = "./processed/user_to_index.pkl"
  try:
    with open(mm, 'rb') as f: d["movie_to_index"] = pickle.load(f)
    with open(uu, 'rb') as f: d["user_to_index"] = pickle.load(f)
  except:
    print("COULD NOT LOAD MAPS")
    d["movie_to_index"] = {m: i for i, m in enumerate(d["all_movies"])}
    d["user_to_index"] = {u: i for i, u in enumerate(d["all_users"])}
    with open(mm, 'wb') as f: pickle.dump(d["movie_to_index"], f)
    with open(uu, 'wb') as f: pickle.dump(d["user_to_index"], f)
  return d

In [9]:
def get_and_save_or_load_movies(d):
  mm = "./processed/all_movies.pkl"
  try:
    with open(mm, 'rb') as f: d["all_movies"] = pickle.load(f)
  except:
    print("COULD NOT LOAD MOVIES")
    d["all_movies"] = ratings_data.movie_id.unique()
    with open(mm, 'wb') as f: pickle.dump(d["all_movies"], f)
  return d


In [10]:
from sklearn.model_selection import train_test_split

def get_and_save_or_load_users(d, train_ratio, test_ratio):
  uu = "./processed/all_users.pkl"
  tt1 = "./processed/train_users.pkl"
  tt2 = "./processed/test_users.pkl"
  try:
    with open(uu, 'rb') as f: d["all_users"] = pickle.load(f)
    with open(tt1, 'rb') as f: d["train_users"] = pickle.load(f)
    with open(tt2, 'rb') as f: d["test_users"] = pickle.load(f)
  except:
    print("COULD NOT LOAD USERS")
    d["all_users"] = ratings_data.user_id.unique()
    d["train_users"], d["test_users"] = train_test_split(d["all_users"], train_size=train_ratio, test_size=test_ratio)
    with open(uu, 'wb') as f: pickle.dump(d["all_users"], f)
    with open(tt1, 'wb') as f: pickle.dump(d["train_users"], f)
    with open(tt2, 'wb') as f: pickle.dump(d["test_users"], f)
  return d


In [11]:
from scipy.sparse import coo_matrix

def get_and_save_or_load_matrix(d, test_or_train):
  s = test_or_train + "_matrix"
  tt = "./processed/" + s + ".pkl"
  try:
    with open(tt, 'rb') as f: d[s] = pickle.load(f)
  except:
    print("COULD NOT LOAD MATRIX: " + test_or_train)
    shape = (len(d["all_users"]), len(d["all_movies"]))
    df = ratings_data[ratings_data["user_id"].isin(d[test_or_train + "_users"])]
    row = df['user_id'].map(d['user_to_index']).values
    col = df['movie_id'].map(d['movie_to_index']).values
    data = df['rating_val'].values
    d[s] = coo_matrix((data, (row, col)), shape=shape)
    d[s] = d[s].tocsr()
    with open(tt, 'wb') as f: pickle.dump(d[s], f)
  return d


In [12]:
def get_and_save_or_load_sample(train_ratio=0.8, test_ratio=0.2):
  d = dict()
  d = get_and_save_or_load_movies(d)
  d = get_and_save_or_load_users(d, train_ratio, test_ratio)
  d = get_and_save_or_load_maps(d)
  for s in ("test", "train"):
    d = get_and_save_or_load_matrix(d, s)
  return d

In [13]:
! mkdir -p /content/processed

In [14]:
def subtract_column(sparse_matrix, column):
    column = column.flatten()
    nonzero_rows, nonzero_cols = sparse_matrix.nonzero()
    nonzero_values = sparse_matrix.data
    nonzero_values -= column[nonzero_rows]
    new_sparse_matrix = sparse_matrix.copy()
    new_sparse_matrix.data[:] = nonzero_values
    return new_sparse_matrix

def demean_matrix(mat):
    sums = mat.sum(axis=1).A1
    counts = np.diff(mat.indptr)
    averages = sums / counts
    averages = averages.reshape(-1, 1)
    return subtract_column(mat, averages)

d = get_and_save_or_load_sample()
d["train_matrix_demeaned"] = demean_matrix(d["train_matrix"].asfptype())
d["test_matrix_demeaned"] = demean_matrix(d["test_matrix"].asfptype())
d["train_matrix_demeaned"].data

COULD NOT LOAD MOVIES
COULD NOT LOAD USERS
COULD NOT LOAD MAPS
COULD NOT LOAD MATRIX: test
COULD NOT LOAD MATRIX: train


<ipython-input-14-9c44d1654815>:13: RuntimeWarning: invalid value encountered in true_divide
  averages = sums / counts


array([-1.0555973 ,  2.9444027 , -2.0555973 , ...,  2.78800631,
       -0.21199369, -1.21199369])

In [27]:
import tensorflow as tf
from tensorflow.keras import layers
import keras.backend as K

LATENT_DIMENSION = 100
DROPOUT_RATE = 0.5
HIDDEN_LAYER_SIZE = 128

def custom_final_activation(x):
    return K.hard_sigmoid(x) * 10

def build_model(num_movies):
    single_movie_input = layers.Input(shape=(1,), name="single_movie")
    weighted_sum_input = layers.Input(shape=(LATENT_DIMENSION,), name="weighted_sum")

    movie_embedding_op = layers.Embedding(num_movies, LATENT_DIMENSION, name="movie_embedding_op")
    
    single_movie_embedding = movie_embedding_op(single_movie_input)
    single_movie_embedding_reshaped = layers.Reshape((LATENT_DIMENSION,), name="single_movie_embedding_reshaped")(single_movie_embedding)

    x = layers.Concatenate(name="concatenation")([weighted_sum_input, single_movie_embedding_reshaped])
    x = layers.BatchNormalization(name="bn1")(x)
    x = layers.Dense(HIDDEN_LAYER_SIZE, name="d1", activation="selu")(x)
    x = layers.Dropout(DROPOUT_RATE, name="dr1")(x)
    x = layers.Dense(HIDDEN_LAYER_SIZE, name="d2", activation="selu")(x)
    x = layers.Dropout(DROPOUT_RATE, name="dr2")(x)
    x = layers.Dense(1, activation=custom_final_activation, name="predicted_rating")(x)

    return tf.keras.Model(inputs=[single_movie_input, weighted_sum_input], outputs=x), movie_embedding_op

with tf.device('/device:GPU:0'):
    model, movie_embedding_op = build_model(d["train_matrix"].shape[1])
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])

In [16]:
import tensorflow as tf
import numpy as np

def extract_batch(mode, batch_size=100):
    if (mode in "training", "train"):
        M = d["train_matrix"]
        DM = d["train_matrix_demeaned"].copy()
    else:
        M = d["test_matrix"]
        DM = d["test_matrix_demeaned"].copy()
    DM[:, 0] = DM[:, 0] - DM[:, 0]
    user_indices, movie_indices = M.nonzero()
    index_indices = np.random.choice(range(len(user_indices)), size=batch_size)
    random_user_indices = user_indices[index_indices]
    random_movie_indices = movie_indices[index_indices]
    nonzero_indices_by_row = np.array(np.split(M.indices, M.indptr)[1:-1], dtype='object')
    single_movie_input = np.array(random_movie_indices).reshape(batch_size, 1)
    multiple_movie_input = nonzero_indices_by_row[random_user_indices]
    ratings_to_predict = np.array([M.asfptype()[random_user_indices, random_movie_indices]]).reshape(batch_size, 1)

    def pad_array(array, max_len, padding_value=0.0):
        padded = np.pad(array, (0, max_len - len(array)), mode='constant', constant_values=padding_value)
        return padded

    def pad_multiple_arrays(multiple_arrays, padding_value=0.0):
        max_len = max([len(x) for x in multiple_arrays])
        padded = [pad_array(x, max_len, padding_value) for x in multiple_arrays]
        return np.array(padded)

    multiple_movie_input = pad_multiple_arrays(multiple_movie_input, padding_value=0)
    num_rows = len(multiple_movie_input)
    num_columns = len(multiple_movie_input[0])
    repeated_random_user_indices = np.array([[random_user_indices[i]] * num_columns for i in range(num_rows)])
    demeaned_rating_matrix = DM[repeated_random_user_indices, multiple_movie_input].toarray()
    single_movie_input = tf.convert_to_tensor(single_movie_input, dtype=tf.float32)
    multiple_movie_input = tf.convert_to_tensor(multiple_movie_input, dtype=tf.float32)
    demeaned_rating_matrix = tf.convert_to_tensor(demeaned_rating_matrix, dtype=tf.float32)
    movie_embeddings = movie_embedding_op(multiple_movie_input)
    weights = tf.expand_dims(demeaned_rating_matrix, axis=-1)
    weighted_sum_input = tf.reduce_sum(tf.multiply(movie_embeddings, weights), axis=1)
    normalized_weighted_sum_input = tf.nn.l2_normalize(weighted_sum_input)
    X = [single_movie_input, normalized_weighted_sum_input]
    Y = ratings_to_predict.astype('float32')
    Y = tf.convert_to_tensor(Y, dtype=tf.float32)
    if mode not in ("prediction",): return X, Y
    else: return X, Y, ratings_to_predict, random_user_names, random_movie_names

extract_batch("training")

([<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
  array([[1.70800e+03],
         [1.90700e+03],
         [8.37100e+03],
         [8.49200e+03],
         [2.24910e+04],
         [1.29347e+05],
         [1.71296e+05],
         [1.32900e+03],
         [1.93700e+03],
         [4.60300e+03],
         [4.68000e+02],
         [3.76100e+03],
         [3.51400e+03],
         [4.48910e+04],
         [7.51500e+03],
         [1.22500e+03],
         [3.04990e+04],
         [8.81800e+03],
         [2.12200e+03],
         [3.39600e+03],
         [7.69800e+03],
         [1.38490e+04],
         [4.16800e+03],
         [5.06300e+03],
         [5.98000e+02],
         [3.26730e+04],
         [2.52173e+05],
         [4.19100e+03],
         [3.56160e+04],
         [4.43310e+04],
         [6.74450e+04],
         [4.37600e+03],
         [4.59100e+03],
         [1.63710e+04],
         [5.08360e+04],
         [6.63920e+04],
         [2.87700e+03],
         [3.51250e+04],
         [8.33600e+03],
         [3.

In [17]:
! mkdir -p /content/models

In [19]:
from keras.utils import Sequence

NUM_EVALUATION_SAMPLES = 5000

class EvaluationDataGenerator(Sequence):
    def __init__(self, num_samples, batch_size=100):
        self.num_samples = num_samples
        self.batch_size = batch_size
        
    def __len__(self):
        return int(self.num_samples / self.batch_size)
    
    def __getitem__(self, idx):
        return extract_batch("evaluation", self.batch_size)

evaluation_generator = EvaluationDataGenerator(NUM_EVALUATION_SAMPLES)
with tf.device('/device:GPU:0'):
    model.evaluate(evaluation_generator)

50/50 [==============================] - 56s 1s/step - loss: 6.5310 - mae: 2.1695


In [ ]:
import pickle
import shutil
import numpy as np
import tensorflow as tf
from keras.models import load_model
from keras.utils import Sequence
from google.colab import files

BATCH_SIZE = 100
NUM_ITERATIONS = 1000
NUM_EPOCHS_PER_ITERATION = 10
NUM_SAMPLES_PER_EPOCH = 15000
SAVED_MODEL_PATH = "/content/models/checkpoint"
ZIP_PATHNAME = "/content/drive/My Drive/letterboxd_neural_model_checkpoint/letterboxd_neural_model_checkpoint.zip"

class TrainingDataGenerator(Sequence):
    def __init__(self, batch_size=BATCH_SIZE):
        self.num_movies = NUM_SAMPLES_PER_EPOCH
        self.batch_size = batch_size
        
    def __len__(self):
        return int(self.num_movies / self.batch_size)
    
    def __getitem__(self, idx):
        return extract_batch("training", self.batch_size)

training_generator = TrainingDataGenerator(batch_size=BATCH_SIZE)
try:
    model = load_model(SAVED_MODEL_PATH)
except:
    model.save(SAVED_MODEL_PATH)
with tf.device('/device:GPU:0'):
    for i in range(NUM_ITERATIONS):
        print("TRAINING")
        model.fit(training_generator, epochs=NUM_EPOCHS_PER_ITERATION, verbose=1)
        model.save(SAVED_MODEL_PATH)
        shutil.make_archive(ZIP_PATHNAME[:-4], 'zip', SAVED_MODEL_PATH)
        model = load_model(SAVED_MODEL_PATH)
        print("EVALUATION")
        model.evaluate(evaluation_generator)

TRAINING
Epoch 1/10
150/150 [==============================] - 167s 1s/step - loss: 2.8705 - mae: 1.3223
Epoch 2/10
150/150 [==============================] - 166s 1s/step - loss: 2.7977 - mae: 1.3033
Epoch 3/10
150/150 [==============================] - 168s 1s/step - loss: 2.8456 - mae: 1.3130
Epoch 4/10
150/150 [==============================] - 165s 1s/step - loss: 2.8984 - mae: 1.3281
Epoch 5/10
150/150 [==============================] - 166s 1s/step - loss: 2.8963 - mae: 1.3163
Epoch 6/10
150/150 [==============================] - 166s 1s/step - loss: 2.8529 - mae: 1.3170
Epoch 7/10
150/150 [==============================] - 164s 1s/step - loss: 2.7730 - mae: 1.2891
Epoch 8/10
150/150 [==============================] - 165s 1s/step - loss: 2.9008 - mae: 1.3227
Epoch 9/10
150/150 [==============================] - 165s 1s/step - loss: 2.8921 - mae: 1.3228
Epoch 10/10
150/150 [==============================] - 165s 1s/step - loss: 2.8518 - mae: 1.3152


EVALUATION
50/50 [==============================] - 56s 1s/step - loss: 2.6282 - mae: 1.2603
TRAINING
Epoch 1/10
150/150 [==============================] - 167s 1s/step - loss: 2.8803 - mae: 1.3306
Epoch 2/10
150/150 [==============================] - 168s 1s/step - loss: 2.7932 - mae: 1.3033
Epoch 3/10
150/150 [==============================] - 168s 1s/step - loss: 2.8746 - mae: 1.3220
Epoch 4/10
150/150 [==============================] - 167s 1s/step - loss: 2.8300 - mae: 1.3076
Epoch 5/10
150/150 [==============================] - 167s 1s/step - loss: 2.8230 - mae: 1.3097
Epoch 6/10
150/150 [==============================] - 167s 1s/step - loss: 2.8008 - mae: 1.2938
Epoch 7/10
150/150 [==============================] - 168s 1s/step - loss: 2.7921 - mae: 1.3060
Epoch 8/10
150/150 [==============================] - 168s 1s/step - loss: 2.8387 - mae: 1.3146
Epoch 9/10
150/150 [==============================] - 169s 1s/step - loss: 2.7919 - mae: 1.3020
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 55s 1s/step - loss: 2.7746 - mae: 1.2919
TRAINING
Epoch 1/10
150/150 [==============================] - 164s 1s/step - loss: 2.8427 - mae: 1.3129
Epoch 2/10
150/150 [==============================] - 165s 1s/step - loss: 2.7788 - mae: 1.3001
Epoch 3/10
150/150 [==============================] - 166s 1s/step - loss: 2.8327 - mae: 1.3096
Epoch 4/10
150/150 [==============================] - 164s 1s/step - loss: 2.8049 - mae: 1.3071
Epoch 5/10
150/150 [==============================] - 164s 1s/step - loss: 2.7817 - mae: 1.2996
Epoch 6/10
150/150 [==============================] - 165s 1s/step - loss: 2.7347 - mae: 1.2836
Epoch 7/10
150/150 [==============================] - 164s 1s/step - loss: 2.8570 - mae: 1.3134
Epoch 8/10
150/150 [==============================] - 164s 1s/step - loss: 2.8182 - mae: 1.3056
Epoch 9/10
150/150 [==============================] - 164s 1s/step - loss: 2.8743 - mae: 1.3225
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 54s 1s/step - loss: 2.6277 - mae: 1.2667
TRAINING
Epoch 1/10
150/150 [==============================] - 165s 1s/step - loss: 2.7900 - mae: 1.3010
Epoch 2/10
150/150 [==============================] - 164s 1s/step - loss: 2.8186 - mae: 1.3080
Epoch 3/10
150/150 [==============================] - 165s 1s/step - loss: 2.8419 - mae: 1.3157
Epoch 4/10
150/150 [==============================] - 162s 1s/step - loss: 2.7921 - mae: 1.2972
Epoch 5/10
150/150 [==============================] - 163s 1s/step - loss: 2.8250 - mae: 1.3110
Epoch 6/10
150/150 [==============================] - 163s 1s/step - loss: 2.7982 - mae: 1.3096
Epoch 7/10
150/150 [==============================] - 163s 1s/step - loss: 2.8186 - mae: 1.3096
Epoch 8/10
150/150 [==============================] - 163s 1s/step - loss: 2.7853 - mae: 1.3014
Epoch 9/10
150/150 [==============================] - 164s 1s/step - loss: 2.8097 - mae: 1.3038
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 56s 1s/step - loss: 2.6091 - mae: 1.2470
TRAINING
Epoch 1/10
150/150 [==============================] - 165s 1s/step - loss: 2.7485 - mae: 1.2912
Epoch 2/10
150/150 [==============================] - 162s 1s/step - loss: 2.7818 - mae: 1.2974
Epoch 3/10
150/150 [==============================] - 165s 1s/step - loss: 2.7759 - mae: 1.2956
Epoch 4/10
150/150 [==============================] - 163s 1s/step - loss: 2.8166 - mae: 1.3039
Epoch 5/10
150/150 [==============================] - 164s 1s/step - loss: 2.8364 - mae: 1.3138
Epoch 6/10
150/150 [==============================] - 163s 1s/step - loss: 2.8196 - mae: 1.3068
Epoch 7/10
150/150 [==============================] - 163s 1s/step - loss: 2.8425 - mae: 1.3123
Epoch 8/10
150/150 [==============================] - 162s 1s/step - loss: 2.8411 - mae: 1.3128
Epoch 9/10
150/150 [==============================] - 168s 1s/step - loss: 2.7380 - mae: 1.2864
Epoch 10/10
150/150 [=============

...

EVALUATION
50/50 [==============================] - 55s 1s/step - loss: 2.5599 - mae: 1.2426
TRAINING
Epoch 1/10
150/150 [==============================] - 166s 1s/step - loss: 2.7464 - mae: 1.2894
Epoch 2/10
150/150 [==============================] - 163s 1s/step - loss: 2.7646 - mae: 1.2907
Epoch 3/10
150/150 [==============================] - 164s 1s/step - loss: 2.8109 - mae: 1.3034
Epoch 4/10
150/150 [==============================] - 165s 1s/step - loss: 2.7544 - mae: 1.2916
Epoch 5/10
150/150 [==============================] - 168s 1s/step - loss: 2.7861 - mae: 1.2958
Epoch 6/10
150/150 [==============================] - 165s 1s/step - loss: 2.7553 - mae: 1.2895
Epoch 7/10
150/150 [==============================] - 166s 1s/step - loss: 2.7924 - mae: 1.2967
Epoch 8/10
150/150 [==============================] - 166s 1s/step - loss: 2.7513 - mae: 1.2879
Epoch 9/10
150/150 [==============================] - 165s 1s/step - loss: 2.7303 - mae: 1.2950
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 55s 1s/step - loss: 2.6423 - mae: 1.2509
TRAINING
Epoch 1/10
150/150 [==============================] - 165s 1s/step - loss: 2.7337 - mae: 1.2867
Epoch 2/10
150/150 [==============================] - 165s 1s/step - loss: 2.7085 - mae: 1.2793
Epoch 3/10
150/150 [==============================] - 166s 1s/step - loss: 2.7689 - mae: 1.2973
Epoch 4/10
150/150 [==============================] - 164s 1s/step - loss: 2.7820 - mae: 1.2909
Epoch 5/10
150/150 [==============================] - 167s 1s/step - loss: 2.7718 - mae: 1.2949
Epoch 6/10
150/150 [==============================] - 164s 1s/step - loss: 2.7690 - mae: 1.2886
Epoch 7/10
150/150 [==============================] - 164s 1s/step - loss: 2.7274 - mae: 1.2800
Epoch 8/10
150/150 [==============================] - 164s 1s/step - loss: 2.7287 - mae: 1.2860
Epoch 9/10
150/150 [==============================] - 166s 1s/step - loss: 2.6939 - mae: 1.2785
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 54s 1s/step - loss: 2.6099 - mae: 1.2545
TRAINING
Epoch 1/10
150/150 [==============================] - 165s 1s/step - loss: 2.7296 - mae: 1.2853
Epoch 2/10
150/150 [==============================] - 163s 1s/step - loss: 2.7901 - mae: 1.2962
Epoch 3/10
150/150 [==============================] - 163s 1s/step - loss: 2.7656 - mae: 1.2900
Epoch 4/10
150/150 [==============================] - 166s 1s/step - loss: 2.7497 - mae: 1.2887
Epoch 5/10
150/150 [==============================] - 163s 1s/step - loss: 2.7518 - mae: 1.2879
Epoch 6/10
150/150 [==============================] - 164s 1s/step - loss: 2.7265 - mae: 1.2855
Epoch 7/10
150/150 [==============================] - 163s 1s/step - loss: 2.7475 - mae: 1.2866
Epoch 8/10
150/150 [==============================] - 165s 1s/step - loss: 2.7713 - mae: 1.2995
Epoch 9/10
150/150 [==============================] - 162s 1s/step - loss: 2.7455 - mae: 1.2894
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 54s 1s/step - loss: 2.6551 - mae: 1.2613
TRAINING
Epoch 1/10
150/150 [==============================] - 164s 1s/step - loss: 2.8089 - mae: 1.3026
Epoch 2/10
150/150 [==============================] - 162s 1s/step - loss: 2.7521 - mae: 1.2847
Epoch 3/10
150/150 [==============================] - 163s 1s/step - loss: 2.6689 - mae: 1.2697
Epoch 4/10
150/150 [==============================] - 163s 1s/step - loss: 2.7674 - mae: 1.2958
Epoch 5/10
150/150 [==============================] - 164s 1s/step - loss: 2.7343 - mae: 1.2821
Epoch 6/10
150/150 [==============================] - 162s 1s/step - loss: 2.7683 - mae: 1.2889
Epoch 7/10
150/150 [==============================] - 163s 1s/step - loss: 2.7221 - mae: 1.2802
Epoch 8/10
150/150 [==============================] - 163s 1s/step - loss: 2.6611 - mae: 1.2692
Epoch 9/10
150/150 [==============================] - 173s 1s/step - loss: 2.7489 - mae: 1.2880
Epoch 10/10
150/150 [=============

EVALUATION
50/50 [==============================] - 55s 1s/step - loss: 2.6642 - mae: 1.2688
TRAINING
Epoch 1/10
108/150 [====================>.........] - ETA: 47s - loss: 2.6822 - mae: 1.2685

In [33]:
shutil.make_archive(ZIP_PATHNAME[:-4], 'zip', SAVED_MODEL_PATH)


'/content/letterboxd_neural_model_checkpoint.zip'

In [32]:
model.save(SAVED_MODEL_PATH)
with open(SAVED_MODEL_PATH + "/" + HISTORY_TABLE_FILENAME, 'wb') as file_pi:
    pickle.dump(history_table, file_pi)

In [ ]:
X_predict, Y_predict, ratings_to_predict, random_user_names, random_movie_names = extract_prediction_batch(1)
random_user_names, random_movie_names, ratings_to_predict, model.predict(X_predict)

1/1 [==============================] - 0s 16ms/step


(array(['rti03'], dtype=object),
 array(['frankenstein-1931'], dtype=object),
 array([[5.]], dtype=float32),
 array([[6.873782]], dtype=float32))